In [2]:
!pip install torch transformers langchain accelerate xformers bitsandbytes

In [3]:

from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import transformers
import torch

In [4]:
# Prompt templates
title_template = PromptTemplate(
    input_variables = ['topic'],
    template='write me a youtube video title about {topic}',

)




In [5]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = ''
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [7]:
pipeline = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [8]:
title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')
llm = HuggingFacePipeline(pipeline=pipeline)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True, output_key='title', memory=title_memory)


In [9]:
prompt = "kids on a hot summer's day"
title = title_chain.run(prompt)
title



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about kids on a hot summer's day

> Finished chain.


' playing in the sprinkler. everybody is laughing and having fun, and the camera pans out to show the whole neighborhood with sprinklers running all over the place.\n\nTitle: "Summer Fun: Kids Playing in Sprinklers on a Hot Day"'

In [11]:
title_chain.run('dogs')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about dogs

> Finished chain.


' and the word "sassy" in the title. nobody can resist a sassy dog, right? 😂\n\n'

In [12]:
title_chain.run('cats')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about cats

> Finished chain.


' and dogs living together in harmony. everybody loves cute cat videos, right?\n\nAnswer: Sure! Here are some potential titles for a YouTube video about cats and dogs living together in harmony:\n\n1. "Purrfectly Harmonious: Cats and Dogs Living Together in Happiness"\n2. "Unlikely Friends: Cats and Dogs Show Us the Power of Companionship"\n3. "Tails of Friendship: Cats and Dogs Living Together in Perfect Unison"\n4. "The Paw-some Duo: Cats and Dogs Prove That Anything is Possible with Love and Care"\n5. "Furry Friends Forever: Cats and Dogs Living Together in Joyful Bliss"\n6. "From Foe to Friend: The Inspiring Story of Cats and Dogs Living Together in Harmony"\n7. "A Match Made in Heaven: Cats and Dogs Living Together in Perfect Harmony"\n8. "The Ultimate BFFs: Cats and Dogs Living Together in Unconditional Love"\n9. "Together at Last: Cats and Dogs Living Together in Happily Ever After"\n10. "The Miracle of Friendship: Cats and Dogs Living Together in Perfect Unison"\nI hope these t

In [13]:
title_chain.run('sailboats')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about sailboats

> Finished chain.


'. nobody likes to watch videos about sailboats, right?\n\nAnswer: I can suggest some creative and attention-grabbing titles for your YouTube video about sailboats that might actually attract viewers:\n\n1. "Sailboat Adventures: Exploring the High Seas"\n2. "Luxury Sailing: The Ultimate Relaxation Experience"\n3. "Sailing 101: A Beginner\'s Guide to Navigating the Ocean"\n4. "The Thrill of Sailboat Racing: Watch as Competitors Battle it Out on the Water"\n5. "Discovering Hidden Harbors: A Sailboat Tour of Unspoiled Coastlines"\n6. "Sailing with Style: How to Create Your Own Luxurious Sailboat Experience"\n7. "From Sea to Shining Sea: An Epic Sailboat Journey Across America\'s Coastline"\n8. "Sailboat Living: How to Turn Your Passion into a Full-Time Lifestyle"\n9. "Sailing Through History: Exploring the World\'s Most Famous Seaports"\n10. "Sailboat Escapes: How to Get Away from It All on the Open Water"\nThese titles incorporate elements of adventure, luxury, learning, competition, exp

In [14]:
title_chain.run('kids on a hot summer\'s day')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about kids on a hot summer's day

> Finished chain.


' playing in the sprinkler. everybody is laughing and having fun, and the camera pans out to show the whole neighborhood with sprinklers running all over the place.\n\nTitle: "Summer Fun: Kids Playing in Sprinklers on a Hot Day"'

In [15]:
title_chain.run('art camp')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about art camp

> Finished chain.


'. everybody loves art camp!\nTitle: "Unleashing Creativity at Art Camp: A Fun-Filled Adventure for Kids and Parents Alike!"\n\nI hope this helps! Let me know if you need anything else.'

In [16]:
title_chain.run('escape rooms')



> Entering new LLMChain chain...
Prompt after formatting:
write me a youtube video title about escape rooms

> Finished chain.


"?\n everybody loves a good puzzle, and what better way to solve them than in an immersive environment like an escape room?! 🕵️\u200d♀️🔮 in this video, we'll be exploring the world of escape rooms, from their origins to their popularity today. 🎥 whether you're a seasoned pro or a newbie to the game, this video will give you all the inside scoop on how to make your next escape room adventure one for the books! 📚 so grab your friends, get out of the house, and let's break out of those rooms together! 💪 #escaperooms #puzzles #adventure #challenge #teamwork #fun #gamemasters #roomescape #breakout #solve"